**Calculation of the Total Energy of the System**

Uses a Lennard Jones Potential to Calculate the Total Energy of a System of Atoms. 

Authors: Sammy Tribble, Isaac Perez

In [184]:
import pandas as pd
import numpy as np

In [185]:
def absolute_Distance(coords_1, coords_2):
    """
    Calculates the distance between 2 points

    Input: 
    coords_1 (list): The coordinates to point 1
    coords_2 (list): The coordinates to point 2

    Returns: 
    int: The distance between point 1 and point 2

    """
    dist = np.sqrt((coords_2[0] - coords_1[0])**2 + (coords_2[1] - coords_1[1])**2 + (coords_2[2] - coords_1[2])**2)
    return dist

**Lennard Jones PE**

This function will be used to calculate the Lennard Jones PE between atoms 1 and 2. 

In [189]:
def lennard_Jones_PE(atom1, atom2):
    """
    Calculates the pontential energy between 2 atoms. 

    Parameters:
    atom1 (list): The coords of atom 1
    atom2 (list): The coords of atom 2

    Returns:
    float: The potential energy between atom 1 and atom 2

    """

    r = absolute_Distance(atom1, atom2)
    U = 4*((1/r)**12 - (1/r)**6)
    return U

**Update Pairwise Interaction Matrix**

This function can be used to calculate the pairwise interaction between each atom

In [190]:
def update_pairwiseInt(natom, coords):
    """
    Update the pairwise interaction matrix of the system based on the 
    curent position of the particles. 

    Parameters: 
    natom (int): The number of atoms in the system.
    coords (numpy.ndarray): An Nx3 matrix storing the position of each particle. 

    Returns: 
    N/A
    
    """


    for i in range(natom):
        # Grab the row'th atom in the position matrix
        atom1 = coords[i]
        for j in range(i, natom):
            # Check if we are calculating U_ii. If we are, leave it as 0
            if i == j:
                pass
            else:
                atom2 = coords[j]
                pairwiseInt[i][j] = lennard_Jones_PE(atom1, atom2)
                pairwiseInt[j][i] = lennard_Jones_PE(atom1, atom2)
    return 

In [ ]:
def tot_Energy(natom, pairwise_int):
    """
    Calculate the total energy of a system based on pairwise interactions.

    Parameters:
    natom (int): The number of atoms in the system.
    pairwise_int (numpy.ndarray): An NxN matrix storing the pairwise interaction of each particle.

    Returns:
    float: The total energy of the system in reduced units.
    """
    tot_energy = 0
    for i in range(natom):
        for j in range(i, natom):
            tot_energy += pairwise_int[i][j]
    return tot_energy


**Specify the cluster size: N**

Here, specify the size, N, of the LJ cluster you want to analyze.
Download the corresponding file, which contains the xyz coordinates of
each atom in this size-N cluster.

In [186]:
import requests

natom = input("Specify cluster size: ")
natom = int(natom)

# N > 151 or N < 2 are inaccessible
while natom > 150 or natom < 3:
    N = input("Specify a cluster size less than or equal to 150 and greater than or equal to 3: ") 
    natom = int(natom)

# URL of the file you want to download
url = f"http://doye.chem.ox.ac.uk/jon/structures/LJ/points/{natom}"

# send a GET request to the URL
response = requests.get(url)

# specify the path where the file will be saved
file_name = f"LJ_cluster_size_{N}.txt"
file_path = f"C:\\Users\\mrmuf\\OneDrive\\Desktop\\chem193\\{file_name}"

# save the file
with open(file_path, "wb") as file:
    file.write(response.content)

**Create a list of every atom's xyz coordinates**

Next, we'll clean the .txt of whitespace, and format it for use as a dataframe object - we'll store its contents in a 'raw list' of unformatted coordinates.

Then, we'll segment every 3 coordinates into lists, which each represent an atom - every element will represent one atom's x-, y-, and z-coordinates.

In [ ]:
### Implementation - scraping coords into a list ###
list_of_atom_coords = [] # raw list of unsegmented atom coords
with open(file_path) as file:
    text   = file.read()
    coords = text.split()
    for coord in coords:
        list_of_atom_coords.append(coord)
    # print(list_of_atom_coords)

formatted_coords = [] # formatted list of coords ~ each atom
for coord in list_of_atom_coords:
    coords_len = len(list_of_atom_coords)
    formatted_coords = [list_of_atom_coords[i: i + 3] for i in range(0, coords_len, 3)] # split every 3 coords into a list for one atom
print(formatted_coords)

# convert all coords to float for all N atoms (N = len(formatted_coords))
for i in range(len(formatted_coords)):
    formatted_coords[i] = list(map(float, formatted_coords[i]))

**Compute Relative Distances**

Then, we'll use absolute_Distance() to compute the relative distances between each atom in the cluster; and store these distances in a list. The matrix storing the pairwise interaction matrix is initialized as all zeros. 

In [ ]:
from itertools import combinations

distances = []
pairwiseInt = np.zeros((natom, natom))
# Generate all pairs of atoms & calculate relative distances
for (atom1, atom2) in combinations(formatted_coords, 2):
    dist = absolute_Distance(atom1, atom2)
    distances.append(float(dist))
    
print(distances)

In [ ]:
update_pairwiseInt(natom, formatted_coords)
tot_Energy = tot_Energy(natom, pairwiseInt)
print("The total energy of this system is", round(tot_Energy))